In [25]:
gameDF = spark.read.csv("/home/training/data/Age-campaign.csv", inferSchema=True, header=True)
gameDF.printSchema()
interDF = gameDF.select("Age","Campaign","Users","Pages / Session","`Avg. Session Duration`","Bounce Rate")
gamesDF = interDF.withColumn("Users", interDF.Users.cast("double"))


root
 |-- Age: string (nullable = true)
 |-- Campaign: string (nullable = true)
 |-- Users: integer (nullable = true)
 |-- New Users: integer (nullable = true)
 |-- Sessions: integer (nullable = true)
 |-- Bounce Rate: string (nullable = true)
 |-- Pages / Session: double (nullable = true)
 |-- Avg. Session Duration: double (nullable = true)
 |-- Goal Conversion Rate: string (nullable = true)
 |-- Goal Completions: integer (nullable = true)
 |-- Goal Value: double (nullable = true)



In [26]:
from pyspark.sql import functions as fn
df1 = gamesDF.withColumnRenamed('Pages / Session', 'Pages_Session')
df2 = df1.withColumnRenamed('Bounce Rate', 'BounceRate')
df3 =  df2.withColumn('BounceRate', fn.regexp_replace('BounceRate', '%', ''))
clean_df = df3.toDF(*(c.replace('.', '_') for c in df3.columns))
final_df = clean_df.withColumnRenamed("Avg_ Session Duration", "AvgSessDuration")
final_df.show()

df = final_df.withColumn("BounceRate", final_df.BounceRate.cast("double"))
df.printSchema()

+-----+--------------------+-----+-------------+---------------+----------+
|  Age|            Campaign|Users|Pages_Session|AvgSessDuration|BounceRate|
+-----+--------------------+-----+-------------+---------------+----------+
|18-24|           (not set)|161.0|         5.83|         253.91|      0.00|
|25-34|           (not set)|118.0|         8.27|         442.39|      2.22|
|35-44|           (not set)| 99.0|          6.3|          284.1|      2.40|
|45-54|           (not set)| 81.0|         6.02|         397.48|      0.00|
|55-64|           (not set)| 52.0|         5.82|         242.43|      0.00|
|  65+|           (not set)| 36.0|          5.8|         287.75|      0.00|
|18-24|rainbow-girls-nye...| 33.0|         6.14|          118.9|      0.00|
|25-34|rainbow-girls-nye...| 27.0|         5.94|          258.9|      0.00|
|18-24|princess-in-colou...| 25.0|         6.06|         275.23|      3.23|
|18-24|princess-easter-s...| 24.0|          6.0|          396.5|      0.00|
|45-54|princ

In [27]:
import pyspark.sql.functions as fn

df.agg( * [ (1 - (fn.count(c) / fn.count('*')) ).alias(c + '_missing') for c in df.columns] ) \
    .show(vertical=True)

-RECORD 0---------------------------------------
 Age_missing             | 0.025974025974025983 
 Campaign_missing        | 0.025974025974025983 
 Users_missing           | 0.0                  
 Pages_Session_missing   | 0.0                  
 AvgSessDuration_missing | 0.0                  
 BounceRate_missing      | 0.0                  



In [28]:
from pyspark.sql import functions as F

df_n1 = df.fillna("baby-boss-photo-shoot", "Campaign")
df_n2 = df_n1.fillna("18-24", "Age")
df_agg = df_n2.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns])
df_agg.show()

+---+--------+-----+-------------+---------------+----------+
|Age|Campaign|Users|Pages_Session|AvgSessDuration|BounceRate|
+---+--------+-----+-------------+---------------+----------+
|  0|       0|    0|            0|              0|         0|
+---+--------+-----+-------------+---------------+----------+



In [29]:
from pyspark.sql.functions import *
testDF1 = df_n2.na.replace(["(not set)"], ["notknown"], 'Campaign')
tf1 = testDF1.na.replace(["(not set)"], ["notknown"], 'Age')
tf = tf1.na.replace(["notknown"], ["baby-boss-photo-shoot"], 'Campaign')
tf.show()

+-----+--------------------+-----+-------------+---------------+----------+
|  Age|            Campaign|Users|Pages_Session|AvgSessDuration|BounceRate|
+-----+--------------------+-----+-------------+---------------+----------+
|18-24|baby-boss-photo-s...|161.0|         5.83|         253.91|       0.0|
|25-34|baby-boss-photo-s...|118.0|         8.27|         442.39|      2.22|
|35-44|baby-boss-photo-s...| 99.0|          6.3|          284.1|       2.4|
|45-54|baby-boss-photo-s...| 81.0|         6.02|         397.48|       0.0|
|55-64|baby-boss-photo-s...| 52.0|         5.82|         242.43|       0.0|
|  65+|baby-boss-photo-s...| 36.0|          5.8|         287.75|       0.0|
|18-24|rainbow-girls-nye...| 33.0|         6.14|          118.9|       0.0|
|25-34|rainbow-girls-nye...| 27.0|         5.94|          258.9|       0.0|
|18-24|princess-in-colou...| 25.0|         6.06|         275.23|      3.23|
|18-24|princess-easter-s...| 24.0|          6.0|          396.5|       0.0|
|45-54|princ

In [30]:
numerical = ['Users','BounceRate','Pages_Session','AvgSessDuration']
desc= tf.describe(numerical)
desc.show()

+-------+------------------+------------------+------------------+------------------+
|summary|             Users|        BounceRate|     Pages_Session|   AvgSessDuration|
+-------+------------------+------------------+------------------+------------------+
|  count|               539|               539|               539|               539|
|   mean| 48.32282003710575|1.9077736549165116| 5.832708719851575|250.46703153988898|
| stddev|121.37747068158338| 3.376550264463662|1.7838340799142733| 173.6730686418285|
|    min|              10.0|               0.0|              2.54|              12.0|
|    max|            1149.0|             37.93|             19.17|           1393.13|
+-------+------------------+------------------+------------------+------------------+



In [31]:
tf.agg( {'BounceRate': 'skewness', 'Users': 'skewness', 'AvgSessDuration': 'skewness'} ).show()

+--------------------+------------------+-------------------------+
|skewness(BounceRate)|   skewness(Users)|skewness(AvgSessDuration)|
+--------------------+------------------+-------------------------+
|    3.59124565265571|6.2029515117970435|       1.8005616542779674|
+--------------------+------------------+-------------------------+



In [32]:
n_numerical = len(numerical)
corr = []

for i in range(0, n_numerical):
    temp = [None] * i
    for j in range(i, n_numerical):
        temp.append(tf.corr(numerical[i], numerical[j]))
    corr.append(temp)
    from tabulate import tabulate
print(tabulate(corr, headers=numerical, showindex=numerical, tablefmt="fancy_grid", numalign="center"))

╒═════════════════╤═════════╤══════════════╤═════════════════╤═══════════════════╕
│                 │  Users  │  BounceRate  │  Pages_Session  │  AvgSessDuration  │
╞═════════════════╪═════════╪══════════════╪═════════════════╪═══════════════════╡
│ Users           │    1    │  0.00231363  │    0.0667305    │     0.0671462     │
├─────────────────┼─────────┼──────────────┼─────────────────┼───────────────────┤
│ BounceRate      │         │      1       │    -0.140196    │    -0.0750716     │
├─────────────────┼─────────┼──────────────┼─────────────────┼───────────────────┤
│ Pages_Session   │         │              │        1        │     0.774263      │
├─────────────────┼─────────┼──────────────┼─────────────────┼───────────────────┤
│ AvgSessDuration │         │              │                 │         1         │
╘═════════════════╧═════════╧══════════════╧═════════════════╧═══════════════════╛


In [33]:
quantiles = tf.approxQuantile("AvgSessDuration", [0.25, 0.75], 0.05)
IQR = quantiles[1] - quantiles[0]
bounds = [quantiles[0] - 1.5 * IQR, quantiles[1] + 1.5 * IQR]
tf1 = tf.where( (tf.BounceRate > bounds[0]) | (tf.BounceRate < bounds[1]) )
tf1.show()

+-----+--------------------+-----+-------------+---------------+----------+
|  Age|            Campaign|Users|Pages_Session|AvgSessDuration|BounceRate|
+-----+--------------------+-----+-------------+---------------+----------+
|18-24|baby-boss-photo-s...|161.0|         5.83|         253.91|       0.0|
|25-34|baby-boss-photo-s...|118.0|         8.27|         442.39|      2.22|
|35-44|baby-boss-photo-s...| 99.0|          6.3|          284.1|       2.4|
|45-54|baby-boss-photo-s...| 81.0|         6.02|         397.48|       0.0|
|55-64|baby-boss-photo-s...| 52.0|         5.82|         242.43|       0.0|
|  65+|baby-boss-photo-s...| 36.0|          5.8|         287.75|       0.0|
|18-24|rainbow-girls-nye...| 33.0|         6.14|          118.9|       0.0|
|25-34|rainbow-girls-nye...| 27.0|         5.94|          258.9|       0.0|
|18-24|princess-in-colou...| 25.0|         6.06|         275.23|      3.23|
|18-24|princess-easter-s...| 24.0|          6.0|          396.5|       0.0|
|45-54|princ

In [34]:

import pyspark.ml.feature as ft

town_indexer = ft.StringIndexer(inputCol="Campaign", outputCol="campaign-num",handleInvalid = "keep")
age_indexer = ft.StringIndexer(inputCol="Age", outputCol="age-num")
town_encoder = ft.OneHotEncoder(inputCol="campaign-num", outputCol='campaign-vec')
age_encoder = ft.OneHotEncoder(inputCol="age-num", outputCol='age-vec')
featuresCreator = ft.VectorAssembler(inputCols=["campaign-vec","age-vec"], outputCol='features-vec')
scaler = ft.StandardScaler(inputCol="features-vec", outputCol="features")

In [36]:

import pyspark.ml.regression as reg
lr = reg.GeneralizedLinearRegression(labelCol= "BounceRate")

In [37]:
from pyspark.ml import Pipeline
pipeline = Pipeline( stages = [age_indexer,age_encoder,town_indexer,town_encoder,featuresCreator,scaler, lr] )

In [38]:
auto_train, auto_test = tf1.randomSplit([0.75, 0.25], seed=1111)
auto_train.persist()

DataFrame[Age: string, Campaign: string, Users: double, Pages_Session: double, AvgSessDuration: double, BounceRate: double]

In [39]:
model = pipeline.fit(auto_train)
prediction = model.transform(auto_test)

In [40]:
import pyspark.ml.evaluation as ev
predictions = model.transform(auto_test)


In [41]:
evaluator = ev.RegressionEvaluator(predictionCol="prediction", labelCol="BounceRate")
print(evaluator.evaluate(predictions, {evaluator.metricName: 'rmse'}))
print(evaluator.evaluate(predictions, {evaluator.metricName: 'r2'}))

3.1540716435087535
-0.1287305533821872


In [42]:
schema = ["Campaign", "Age"]
test_record = [ ( "adorable-girls-valentino-fashion", "45-54" ) ]
df = spark.createDataFrame(test_record, schema=schema)
model.transform(df).first().prediction

2.060612142182409